In [ ]:
# needed libraries: gensim & pyhton-Levenshtein

In [3]:
import pandas as pd
import numpy as np
import gensim
import string

In [17]:
def read_data (path):
    df = pd.read_csv(path)
    return df
#     df = pd.read_csv('train.csv')
    # print(df)
    # print(type(df))

In [5]:
def join_question_data(data):
    data = df["question1"].values.tolist()
    data.extend(df["question2"].values.tolist())
    # print(len(data))
    # print(data[0])
    return data

In [6]:
# preprocessing the data (tokenizing and removing punctutation)
def gensim_preprocess(question_data):
    new_data = [gensim.utils.simple_preprocess(str(sentence)) for sentence in question_data] 
    return new_data

In [7]:
def string_preprocess(question_data):
    stop_words=['the', 'a', 'an', 'and', 'is', 'be', 'will']
    new_data = []
    
    for question in question_data:
        processed_question = []
        question = str(question)
        
        #split the question to words (doesn't include stop words)
        words = [word.lower() for word in question.split() if word not in stop_words]
        for word in words:
            # to remove punctutation from string (imported from string library)
            word = word.translate(str.maketrans('', '', string.punctuation))
            word = word.replace('“', '').replace('”', '')

            if len(word) > 0:
                processed_question.append(word)
                
        new_data.append(processed_question)
    
    return new_data

In [26]:
# default vector_size is 100
def gensim_word2vec(training_data, window_size, min_word_count, vector_size, workers):
    model = gensim.models.Word2Vec(window=5, vector_size=300, min_count=3, workers = 12)
#     model.build_vocab(new_data, progress_per= 1000)
    model.build_vocab(training_data, progress_per= 1000)
    model.train(training_data, total_examples= model.corpus_count, epochs=model.epochs)
#     print(model.epochs)
#     print(model.corpus_count)
    return model

In [20]:
# model.save("word2vec_model_one_gesim.model")
def save_gensim_model(model, name):
#     model.save("word2vec_model_one_gensim_vector_300.model")
    model.save(str(name) + ".model")

In [21]:
def find_most_similar(model, word:str):
    return model.wv.most_similar(word)

In [31]:
def get_word_embeddings (model ,word:str):
#     model.wv.get_vector[word]
    return model.wv[word]

In [28]:
df = read_data('train.csv')
data = join_question_data(df)

In [29]:
processed_data_manual = string_preprocess(data)
processed_data_gensim = gensim_preprocess(data)

In [30]:
model_1 = gensim_word2vec(processed_data_manual, 5, 2, 100, 12)
model_2 = gensim_word2vec(processed_data_gensim, 5, 2, 100, 12)

In [41]:
save_gensim_model(model_1, "word2vec_model_manual_100")
save_gensim_model(model_2, "word2vec_model_gensim_100")

In [35]:
print(find_most_similar(model_1, "what"))

[('which', 0.4715254306793213), ('whats', 0.443397581577301), ('horticulture', 0.36997199058532715), ('what’s', 0.36587104201316833), ('proverbs', 0.36556464433670044), ('whatre', 0.35183969140052795), ('some', 0.35031643509864807), ('insights', 0.3402627408504486), ('practices', 0.34008562564849854), ('superstitions', 0.33570602536201477)]


In [36]:
print(find_most_similar(model_2, "what"))

[('which', 0.521908700466156), ('the', 0.47264647483825684), ('exersice', 0.3212506175041199), ('some', 0.3124181628227234), ('productivity', 0.3114285469055176), ('seo', 0.30419066548347473), ('grails', 0.30178216099739075), ('horror', 0.3007189631462097), ('whta', 0.2945222854614258), ('shayari', 0.2923395037651062)]


In [37]:
print(processed_data_manual[0:5])

[['what', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], ['what', 'story', 'of', 'kohinoor', 'kohinoor', 'diamond'], ['how', 'can', 'i', 'increase', 'speed', 'of', 'my', 'internet', 'connection', 'while', 'using', 'vpn'], ['why', 'am', 'i', 'mentally', 'very', 'lonely', 'how', 'can', 'i', 'solve', 'it'], ['which', 'one', 'dissolve', 'in', 'water', 'quikly', 'sugar', 'salt', 'methane', 'carbon', 'di', 'oxide']]


In [38]:
print(processed_data_gensim[0:5])

[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], ['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh', 'noor', 'diamond'], ['how', 'can', 'increase', 'the', 'speed', 'of', 'my', 'internet', 'connection', 'while', 'using', 'vpn'], ['why', 'am', 'mentally', 'very', 'lonely', 'how', 'can', 'solve', 'it'], ['which', 'one', 'dissolve', 'in', 'water', 'quikly', 'sugar', 'salt', 'methane', 'and', 'carbon', 'di', 'oxide']]
